<a href="https://colab.research.google.com/github/YolandaMDavis/cs109a-fall2020/blob/covid-research/New_York_Covid_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# How to use Geohash - https://bigfastblog.com/geohash-intro

!pip install pygeohash

In [3]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pygeohash as pgh
DATA_ROOT = '/content/drive/My Drive/Final-Project-for-CS109a/covid_analysis/'

In [4]:
new_york_covid_df = pd.read_csv(DATA_ROOT + 'nyc_covid.csv')
new_york_covid_df.head()

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,491,2083.77,23563.03,28,118.83,4.55,10794
1,10002,Chinatown/Lower East Side,Manhattan,1497,1950.35,76755.41,160,208.45,5.71,26230
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,747,1388.43,53801.62,35,65.05,2.72,27421
3,10004,Financial District,Manhattan,58,1588.78,3650.61,1,27.39,3.79,1529
4,10005,Financial District,Manhattan,122,1453.05,8396.11,2,23.82,3.60,3386


In [5]:
new_york_covid_totals_df = new_york_covid_df.groupby("BOROUGH_GROUP").sum()[["COVID_CASE_COUNT","TOTAL_COVID_TESTS"]].reset_index()
new_york_covid_totals_df["PERCENT_POSITIVE_TESTS"] = new_york_covid_totals_df["COVID_CASE_COUNT"] / new_york_covid_totals_df["TOTAL_COVID_TESTS"]
new_york_covid_totals_df["BOROUGH_GROUP"] = new_york_covid_totals_df["BOROUGH_GROUP"].str.upper()
new_york_covid_totals_df = new_york_covid_totals_df.rename(columns={"BOROUGH_GROUP": "BOROUGH"})
new_york_covid_totals_df


,BOROUGH,COVID_CASE_COUNT,TOTAL_COVID_TESTS,PERCENT_POSITIVE_TESTS
0,BRONX,55198,489908,0.112670
1,BROOKLYN,71945,848647,0.084776
2,MANHATTAN,32900,649449,0.050658
3,QUEENS,75450,737985,0.102238
4,STATEN ISLAND,17120,160932,0.106380


In [6]:
new_york_crime_df = pd.read_csv(DATA_ROOT + 'nyc_crime.csv')
new_york_crime_df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,972326799,81,NaN,09/28/2020,21:27:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/28/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK,M,1003904,186483,40.678516,-73.929143,"(40.67851591200008, -73.92914304899993)",POINT (-73.92914304899993 40.67851591200008)
1,376304873,52,NaN,09/27/2020,19:13:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/27/2020,NaN,45-64,WHITE HISPANIC,M,NaN,UNKNOWN,BLACK HISPANIC,M,1017928,258050,40.874906,-73.878224,"(40.87490600500007, -73.87822380899996)",POINT (-73.87822380899996 40.87490600500007)
2,299326203,75,NaN,09/21/2020,01:21:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/21/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK,M,1021234,181211,40.663990,-73.866692,"(40.66399002800006, -73.86669235099998)",POINT (-73.86669235099998 40.66399002800006)
3,674946147,121,NaN,09/15/2020,08:46:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/15/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK,M,938430,170972,40.635845,-74.165090,"(40.63584491100005, -74.165090337)",POINT (-74.165090337 40.63584491100005)
4,416422620,101,NaN,09/08/2020,13:50:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/08/2020,NaN,<18,BLACK,M,NaN,18-24,BLACK,M,1049837,157548,40.598875,-73.763823,"(40.59887464700005, -73.76382298499993)",POINT (-73.76382298499993 40.59887464700005)


In [7]:
new_york_crime_df.shape

(306656, 36)

In [8]:
new_york_crime_df['BORO_NM'].unique()

array([nan, 'BRONX', 'MANHATTAN', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'],
      dtype=object)

In [9]:
# select only 2020 data and drop rows that have na for offenses and boroughs

new_york_crime_df = new_york_crime_df[new_york_crime_df['OFNS_DESC'].notna() & new_york_crime_df['BORO_NM'].notna() ]
new_york_crime_df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
12,843609180,43,BRONX,08/24/2020,20:57:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,578,VIOLATION,NaN,HARRASSMENT 2,NaN,PATROL BORO BRONX,638.0,"HARASSMENT,SUBD 3,4,5",STREET,08/24/2020,NaN,NaN,NaN,NaN,NaN,45-64,WHITE HISPANIC,M,1020906,242723,40.832826,-73.867539,"(40.83282612800008, -73.867539358)",POINT (-73.867539358 40.83282612800008)
18,774311943,52,BRONX,08/12/2020,00:05:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,118,FELONY,NaN,DANGEROUS WEAPONS,NaN,PATROL BORO BRONX,793.0,WEAPONS POSSESSION 3,STREET,08/12/2020,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,1012928,255180,40.867046,-73.896316,"(40.86704639800007, -73.89631577699998)",POINT (-73.89631577699998 40.86704639800007)
32,215695380,44,BRONX,06/06/2020,00:10:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,118,FELONY,NaN,DANGEROUS WEAPONS,NaN,PATROL BORO BRONX,793.0,WEAPONS POSSESSION 3,STREET,06/06/2020,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,1004680,239996,40.825394,-73.926183,"(40.825393733000055, -73.92618276999998)",POINT (-73.92618276999998 40.825393733000055)
34,960386249,13,MANHATTAN,05/31/2020,00:05:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,114,FELONY,NaN,ARSON,NaN,PATROL BORO MAN SOUTH,264.0,"ARSON, MOTOR VEHICLE 1 2 3 & 4",STREET,05/31/2020,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,988809,206788,40.734269,-73.983551,"(40.734268996000026, -73.98355117399994)",POINT (-73.98355117399994 40.734268996000026)
36,398930216,1,MANHATTAN,05/18/2020,00:15:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,104,FELONY,NaN,RAPE,NaN,PATROL BORO MAN SOUTH,157.0,RAPE 1,STREET,05/18/2020,NaN,25-44,UNKNOWN,M,NaN,25-44,BLACK,F,982285,201682,40.720255,-74.007090,"(40.72025522300004, -74.00709027999993)",POINT (-74.00709027999993 40.72025522300004)


In [10]:
# convert to datetime 

ny_datetime =  pd.DatetimeIndex(new_york_crime_df['RPT_DT'])
ny_time = pd.DatetimeIndex(new_york_crime_df['CMPLNT_FR_TM'])
new_york_crime_df['YEAR'] = ny_datetime.year
new_york_crime_df['MONTH'] = ny_datetime.month
new_york_crime_df['DAY'] = ny_datetime.day
new_york_crime_df['DAY_OF_WEEK'] = ny_datetime.day_name()
new_york_crime_df['HOUR'] = ny_time.hour
new_york_crime_df.head()


,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,YEAR,MONTH,DAY,DAY_OF_WEEK,HOUR
12,843609180,43,BRONX,08/24/2020,20:57:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,578,VIOLATION,NaN,HARRASSMENT 2,NaN,PATROL BORO BRONX,638.0,"HARASSMENT,SUBD 3,4,5",STREET,08/24/2020,NaN,NaN,NaN,NaN,NaN,45-64,WHITE HISPANIC,M,1020906,242723,40.832826,-73.867539,"(40.83282612800008, -73.867539358)",POINT (-73.867539358 40.83282612800008),2020,8,24,Monday,20
18,774311943,52,BRONX,08/12/2020,00:05:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,118,FELONY,NaN,DANGEROUS WEAPONS,NaN,PATROL BORO BRONX,793.0,WEAPONS POSSESSION 3,STREET,08/12/2020,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,1012928,255180,40.867046,-73.896316,"(40.86704639800007, -73.89631577699998)",POINT (-73.89631577699998 40.86704639800007),2020,8,12,Wednesday,0
32,215695380,44,BRONX,06/06/2020,00:10:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,118,FELONY,NaN,DANGEROUS WEAPONS,NaN,PATROL BORO BRONX,793.0,WEAPONS POSSESSION 3,STREET,06/06/2020,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,1004680,239996,40.825394,-73.926183,"(40.825393733000055, -73.92618276999998)",POINT (-73.92618276999998 40.825393733000055),2020,6,6,Saturday,0
34,960386249,13,MANHATTAN,05/31/2020,00:05:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,114,FELONY,NaN,ARSON,NaN,PATROL BORO MAN SOUTH,264.0,"ARSON, MOTOR VEHICLE 1 2 3 & 4",STREET,05/31/2020,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,988809,206788,40.734269,-73.983551,"(40.734268996000026, -73.98355117399994)",POINT (-73.98355117399994 40.734268996000026),2020,5,31,Sunday,0
36,398930216,1,MANHATTAN,05/18/2020,00:15:00,NaN,NaN,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,104,FELONY,NaN,RAPE,NaN,PATROL BORO MAN SOUTH,157.0,RAPE 1,STREET,05/18/2020,NaN,25-44,UNKNOWN,M,NaN,25-44,BLACK,F,982285,201682,40.720255,-74.007090,"(40.72025522300004, -74.00709027999993)",POINT (-74.00709027999993 40.72025522300004),2020,5,18,Monday,0


In [11]:
# select only 2020 data and rename columns
new_york_crime_2020_df = new_york_crime_df[new_york_crime_df['YEAR'] == 2020]
new_york_crime_2020_df = new_york_crime_2020_df[["BORO_NM","CMPLNT_FR_DT","CMPLNT_FR_TM","LAW_CAT_CD","OFNS_DESC","PD_DESC","RPT_DT","Latitude","Longitude","YEAR","MONTH","DAY","DAY_OF_WEEK","HOUR"]] 
new_york_crime_2020_df = new_york_crime_2020_df.rename(columns={"BORO_NM": "BOROUGH","OFNS_DESC":"OFFENSE_DESCRIPTION"})
new_york_crime_2020_df.head()

,BOROUGH,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFFENSE_DESCRIPTION,PD_DESC,RPT_DT,Latitude,Longitude,YEAR,MONTH,DAY,DAY_OF_WEEK,HOUR
12,BRONX,08/24/2020,20:57:00,VIOLATION,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",08/24/2020,40.832826,-73.867539,2020,8,24,Monday,20
18,BRONX,08/12/2020,00:05:00,FELONY,DANGEROUS WEAPONS,WEAPONS POSSESSION 3,08/12/2020,40.867046,-73.896316,2020,8,12,Wednesday,0
32,BRONX,06/06/2020,00:10:00,FELONY,DANGEROUS WEAPONS,WEAPONS POSSESSION 3,06/06/2020,40.825394,-73.926183,2020,6,6,Saturday,0
34,MANHATTAN,05/31/2020,00:05:00,FELONY,ARSON,"ARSON, MOTOR VEHICLE 1 2 3 & 4",05/31/2020,40.734269,-73.983551,2020,5,31,Sunday,0
36,MANHATTAN,05/18/2020,00:15:00,FELONY,RAPE,RAPE 1,05/18/2020,40.720255,-74.007090,2020,5,18,Monday,0


In [12]:
new_york_crime_covid_df  =  pd.merge(new_york_crime_2020_df,new_york_covid_totals_df, left_on="BOROUGH", right_on="BOROUGH")
new_york_crime_covid_df

,BOROUGH,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFFENSE_DESCRIPTION,PD_DESC,RPT_DT,Latitude,Longitude,YEAR,MONTH,DAY,DAY_OF_WEEK,HOUR,COVID_CASE_COUNT,TOTAL_COVID_TESTS,PERCENT_POSITIVE_TESTS
0,BRONX,08/24/2020,20:57:00,VIOLATION,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",08/24/2020,40.832826,-73.867539,2020,8,24,Monday,20,55198,489908,0.11267
1,BRONX,08/12/2020,00:05:00,FELONY,DANGEROUS WEAPONS,WEAPONS POSSESSION 3,08/12/2020,40.867046,-73.896316,2020,8,12,Wednesday,0,55198,489908,0.11267
2,BRONX,06/06/2020,00:10:00,FELONY,DANGEROUS WEAPONS,WEAPONS POSSESSION 3,06/06/2020,40.825394,-73.926183,2020,6,6,Saturday,0,55198,489908,0.11267
3,BRONX,02/19/2020,02:35:00,FELONY,BURGLARY,"BURGLARY,TRUCK NIGHT",02/19/2020,40.816088,-73.895824,2020,2,19,Wednesday,2,55198,489908,0.11267
4,BRONX,09/30/2020,05:35:00,FELONY,FELONY ASSAULT,ASSAULT POLICE/PEACE OFFICER,09/30/2020,40.883622,-73.857721,2020,9,30,Wednesday,5,55198,489908,0.11267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306283,STATEN ISLAND,01/01/2020,16:00:00,VIOLATION,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",01/01/2020,40.628101,-74.121657,2020,1,1,Wednesday,16,17120,160932,0.10638
306284,STATEN ISLAND,11/30/2019,09:00:00,VIOLATION,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",01/02/2020,40.625701,-74.146365,2020,1,2,Thursday,9,17120,160932,0.10638
306285,STATEN ISLAND,01/01/2020,17:00:00,MISDEMEANOR,PETIT LARCENY,"LARCENY,PETIT FROM BUILDING,UN",01/01/2020,40.560329,-74.163770,2020,1,1,Wednesday,17,17120,160932,0.10638
306286,STATEN ISLAND,12/18/2019,15:00:00,FELONY,GRAND LARCENY,"LARCENY,GRAND BY FALSE PROMISE-IN PERSON CONTACT",01/02/2020,40.630367,-74.111930,2020,1,2,Thursday,15,17120,160932,0.10638


In [13]:
new_york_crime_offense_types_df = pd.read_csv(DATA_ROOT + 'nyc_offense_types.csv')
new_york_crime_offense_types_df

,OFFENSE_DESCRIPTION,VIOLENT_CRIME,OFFENSE_TYPE
0,ADMINISTRATIVE CODE,0,Other
1,AGRICULTURE & MRKTS LAW-UNCLASSIFIED,0,Other
2,ALCOHOLIC BEVERAGE CONTROL LAW,0,Drugs/Alcohol
3,ANTICIPATORY OFFENSES,0,Other
4,ARSON,1,MV Accident/Property Damage
5,ASSAULT 3 & RELATED OFFENSES,1,Assault and/or Battery
6,BURGLAR'S TOOLS,0,Theft/Robbery/B&E
7,BURGLARY,0,Theft/Robbery/B&E
8,CHILD ABANDONMENT/NON SUPPORT,0,Child Abuse/Endangerment
9,CRIMINAL MISCHIEF & RELATED OF,0,Other


In [14]:
new_york_crime_2020_df = pd.merge(new_york_crime_covid_df,new_york_crime_offense_types_df, left_on="OFFENSE_DESCRIPTION", right_on="OFFENSE_DESCRIPTION")
new_york_crime_2020_df.shape

(306288, 19)

In [15]:
new_york_crime_2020_df["GEOHASH"] = [pgh.encode(row["Latitude"],row["Longitude"]) for idx, row in new_york_crime_2020_df.iterrows()]
new_york_crime_2020_df.sort_values(by=["GEOHASH"])

,BOROUGH,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFFENSE_DESCRIPTION,PD_DESC,RPT_DT,Latitude,Longitude,YEAR,MONTH,DAY,DAY_OF_WEEK,HOUR,COVID_CASE_COUNT,TOTAL_COVID_TESTS,PERCENT_POSITIVE_TESTS,VIOLENT_CRIME,OFFENSE_TYPE,GEOHASH
91252,STATEN ISLAND,09/07/2020,16:03:00,MISDEMEANOR,OFF. AGNST PUB ORD SENSBLTY &,AGGRAVATED HARASSMENT 2,09/07/2020,40.502462,-74.253257,2020,9,7,Monday,16,17120,160932,0.10638,0,Other,dr5nqm4rvwnm
270565,STATEN ISLAND,07/20/2020,22:00:00,FELONY,CRIMINAL MISCHIEF & RELATED OF,"MISCHIEF,CRIMINAL, UNCL 2ND",07/20/2020,40.502784,-74.251831,2020,7,20,Monday,22,17120,160932,0.10638,0,Other,dr5nqm73yfjg
271986,STATEN ISLAND,12/01/2019,00:00:00,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,"CRIMINAL MISCHIEF 4TH, GRAFFIT",01/01/2020,40.502784,-74.251831,2020,1,1,Wednesday,0,17120,160932,0.10638,0,Other,dr5nqm73yfjg
49403,STATEN ISLAND,06/17/2019,17:00:00,VIOLATION,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",03/04/2020,40.502537,-74.251622,2020,3,4,Wednesday,17,17120,160932,0.10638,0,Harassment,dr5nqm787u4h
288351,STATEN ISLAND,04/02/2020,12:00:00,FELONY,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",04/23/2020,40.506569,-74.254377,2020,4,23,Thursday,12,17120,160932,0.10638,0,Theft/Robbery/B&E,dr5nqmcxgeem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199428,BRONX,07/20/2020,01:00:00,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,OBSTR BREATH/CIRCUL,07/24/2020,40.889176,-73.818120,2020,7,24,Friday,1,55198,489908,0.11267,1,Assault and/or Battery,dr7885xk9nuh
242443,BRONX,02/20/2020,09:00:00,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,"MISCHIEF, CRIMINAL 4, OF MOTOR",02/21/2020,40.890085,-73.819850,2020,2,21,Friday,9,55198,489908,0.11267,0,Other,dr7885y40b60
297929,BRONX,02/22/2020,03:15:00,MISDEMEANOR,INTOXICATED & IMPAIRED DRIVING,"INTOXICATED DRIVING,ALCOHOL",02/22/2020,40.886721,-73.815284,2020,2,22,Saturday,3,55198,489908,0.11267,0,Drugs/Alcohol,dr78871q7jnt
297984,BRONX,01/02/2020,20:15:00,MISDEMEANOR,INTOXICATED & IMPAIRED DRIVING,"INTOXICATED DRIVING,ALCOHOL",01/03/2020,40.886721,-73.815284,2020,1,3,Friday,20,55198,489908,0.11267,0,Drugs/Alcohol,dr78871q7jnt


In [16]:
new_york_crime_2020_df.shape

(306288, 20)